In [6]:
from tl_tools import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import pandas as pd
from tensorflow.keras.applications import ResNet101V2

In [7]:
setup_mixed_precision()
train_dir = '/Users/pimpijnenburg/Desktop/Thesis/USTC_SmokeRS_dataset/data/USTC_SmokeRS/processed/train'

train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True ,vertical_flip=True)
train =train_datagen.flow_from_directory(train_dir, color_mode= 'rgb', batch_size = 16, shuffle= True, seed = 1, target_size=(256, 256))


X_train, y_train, X_val, y_val = train_val_split(train, val_split= 0.3)

Mixed precision policy set to: mixed_float16

Found 4980 images belonging to 6 classes.
Number of batches in the training data: 312
Batch size of a single batch 16
Number of samples in the training dataset 4980

Number of training data batches with val split of 0.3: 219
Number of validation data batches: 93

Shape of image training set: (3504, 256, 256, 3)
Shape of image validation set: (1476, 256, 256, 3)

Shape of label training set: (3504, 6)
Shape of label validation set: (1476, 6)


In [8]:
pooling = 'avg'
resnet101v2 = ResNet101V2(weights= 'imagenet',include_top= False, input_shape= (256, 256, 3), pooling = pooling)
resnet101v2.trainable = False

model = fc_layers(resnet101v2, name = 'resnet101v2')

In [ ]:
model.compile(optimizer=Adam(learning_rate = 0.001, clipnorm= 1.0),
                     loss='categorical_crossentropy', 
                     metrics=['accuracy', 'F1Score'])

early_stopping = EarlyStopping(monitor='val_loss', patience=15,restore_best_weights=True, start_from_epoch=20)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=8, min_lr=1e-6)

history =  model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, verbose=1, 
                            callbacks= [early_stopping, reduce_lr])

Epoch 1/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 70s 558ms/step - F1Score: 0.4668 - accuracy: 0.4734 - loss: 1.5085 - val_F1Score: 0.7376 - val_accuracy: 0.7337 - val_loss: 0.7739 - learning_rate: 0.0010
Epoch 2/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 54s 492ms/step - F1Score: 0.7471 - accuracy: 0.7552 - loss: 0.7035 - val_F1Score: 0.7914 - val_accuracy: 0.7947 - val_loss: 0.5961 - learning_rate: 0.0010
Epoch 3/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 54s 488ms/step - F1Score: 0.8109 - accuracy: 0.8150 - loss: 0.5450 - val_F1Score: 0.7843 - val_accuracy: 0.7818 - val_loss: 0.6019 - learning_rate: 0.0010
Epoch 4/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 53s 486ms/step - F1Score: 0.8267 - accuracy: 0.8305 - loss: 0.5139 - val_F1Score: 0.8235 - val_accuracy: 0.8259 - val_loss: 0.5065 - learning_rate: 0.0010
Epoch 5/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 54s 487ms/step - F1Score: 0.8507 - accuracy: 0.8539 - loss: 0.4489 - val_F1Score: 0.8280 - val_accuracy: 0.8293 - val_loss: 0.4755 - learning_rate: 0.0010
Epoch 6/50
110/110 ━━━━━━

In [10]:
path = '/Users/pimpijnenburg/Desktop/Thesis/USTC_SmokeRS_dataset/code/transfer_learning/trained_models'
model_save_path = os.path.join(path, 'resnet101v2.keras')
model.save(model_save_path) 